In [1]:
import tensorflow as tf
import numpy as np
import dbGeneration as db
import random

# Data Pre-Processing

In [2]:
tensorInputs, raw_labels = db.get_tensorinputs_and_labels(folderName = "3per")
tensorInputs = np.asarray(tensorInputs)


Returned data points in 81 batches


In [3]:
n = 0


for i in tensorInputs:
    n += len(i)
    
print(n, "training samples")

338448 training samples


In [4]:
print(len(raw_labels))

81


In [5]:
print(np.asarray(tensorInputs.shape))

tensorInputs2 = []
raw_labels2 = []

for i in tensorInputs:
    for j in i:
        tensorInputs2.append(j)

for i in raw_labels:
    for j in i:
        raw_labels2.append(j)

print(len(tensorInputs2))
print(len(raw_labels2))

print(tensorInputs2[0])
print(raw_labels2[0])

[81]
338448
338448
[[ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0. -1.  0.  0.  1.  0.  0.  2.]
 [ 2.  0.  0.  0. -1. -1. -1.  0.  0.  2.]
 [ 2.  0.  0.  0.  1. -1. -1.  0.  0.  2.]
 [ 2.  0.  0.  0.  1.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]]
-1


In [6]:
raw_labels = raw_labels2
tensorInputs = tensorInputs2
del(raw_labels2)
del(tensorInputs2)

In [7]:
print(raw_labels[:10])

[-1, -1, -1, 1, 1, 1, 1, 1, 1, 1]


### Convert labels to one-hot vector

In [8]:
pre_labels = []
for i in raw_labels:
    if i == -1:
        pre_labels.append([1,0])
    else:
        pre_labels.append([0,1])

pre_labels = np.asarray(pre_labels)

In [9]:
print(pre_labels.shape)
print(pre_labels)

(338448, 2)
[[1 0]
 [1 0]
 [1 0]
 ..., 
 [1 0]
 [1 0]
 [1 0]]


### Define training and testing data

In [10]:
print(len(tensorInputs))

338448


In [11]:
BATCH_SIZE = 256
N_SAMPLES = len(tensorInputs)
N_TRAIN = 320000
N_VALIDATION = int((N_SAMPLES - N_TRAIN)/2)
N_BATCHES = int(N_TRAIN/BATCH_SIZE)

In [12]:
print(N_BATCHES)

1250


## Shuffle data to avoid grouping among games

In [13]:
def get_randomised_batches(training_data, training_labels):
    pairs = []
    for i in range(len(training_data)):
        pairs.append([training_data[i], training_labels[i]])
        
    random.shuffle(pairs)
    x = []
    y = []

    for i in pairs:
        x.append(np.reshape(i[0], (100)))
        y.append(i[1])
        
    return x, y

In [14]:
train_gameboards = tensorInputs[:N_TRAIN]
validation_gameboards = tensorInputs[N_TRAIN:N_TRAIN+N_VALIDATION]
test_gameboards = tensorInputs[N_TRAIN+N_VALIDATION:]

train_labels = pre_labels[:N_TRAIN]
validation_labels = pre_labels[N_TRAIN:N_TRAIN+N_VALIDATION]
test_labels = pre_labels[N_TRAIN+N_VALIDATION:]

In [15]:
print(len(train_gameboards))
print(len(validation_gameboards))
print(len(test_gameboards))

320000
9224
9224


In [16]:
print(test_gameboards[0], test_labels[0])

[[ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 2.  0.  0.  0.  0.  0.  0.  0.  0.  2.]
 [ 2.  0.  0.  1. -1. -1.  0.  0.  0.  2.]
 [ 2. -1.  1.  1.  1.  1. -1.  0. -1.  2.]
 [ 2.  1.  1.  1. -1.  1.  1. -1.  0.  2.]
 [ 2. -1.  1. -1.  1.  1. -1. -1.  0.  2.]
 [ 2. -1. -1.  1. -1.  1. -1. -1.  0.  2.]
 [ 2.  0.  0.  1.  1. -1.  1.  0.  0.  2.]
 [ 2.  0.  0.  1.  1.  1.  1.  1.  0.  2.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]] [1 0]


# Model Definition

In [17]:
gameboards = tf.placeholder(tf.float32, (None, 100), name="gameBoards")

gameboards2d = tf.reshape(gameboards, (-1, 10, 10, 1), name="gameBoards2d")



In [18]:
# First Convulutional layer, small 2x2 filter
conv1 = tf.layers.conv2d(gameboards2d, 64, 2, padding="same", name="Conv1", activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1, 2, 2, name="Pool1")
conv2 = tf.layers.conv2d(pool1, 128, 3, padding="same", name="Conv2", activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2, 2, 2, name="Pool2")

conv3 = tf.layers.conv2d(pool2, 128, 4, padding="same", name="Conv3", activation=tf.nn.relu)
pool3 = tf.layers.max_pooling2d(conv3, 2, 2, name="Pool3")

# Reshape the 2D tensor back to 1D to be fed into "Dense"
# Flatten out the pooling - GET THIS NUMBER FROM POOLx.SHAPE
pool2_flat = tf.reshape(pool3, (-1, int(1*1*128)), name="Pool2_Flat")


# The dropout allows us to train a subset of the neurons at any given iteration.  
keep_prob = tf.placeholder(tf.float32, name="Keep_Probability")


# A dense layer with dropout
# DENSE - a fully connected linear transofmration of every dimension of the data
dense = tf.layers.dense(pool2_flat, int(128), activation=tf.nn.relu, name="Dense")

# DROPOUT - if set to 0.5, rendomly select 50% of the neurons to ignore (different with each computation)
dropout = tf.nn.dropout(dense, keep_prob, name="Dropout")

dense2 = tf.layers.dense(dropout, int(128), activation=tf.nn.relu, name="Dense2")
dropout2 = tf.nn.dropout(dense2, keep_prob, name="Dropout2")

"""dense3 = tf.layers.dense(dropout, int(128), activation=tf.nn.relu, name="Dense3")
dropout3 = tf.nn.dropout(dense3, keep_prob, name="Dropout3")"""

# A dense layer to classify the final values. Only 2 neurons. 
predictions = tf.layers.dense(dropout2, 2, activation=None, name="Predictions")

In [19]:
total_parameters = 0
for variable in tf.trainable_variables():
    # shape is an array of tf.Dimension
    shape = variable.get_shape()
    print(shape)
    print(len(shape))
    variable_parameters = 1
    for dim in shape:
        print(dim)
        variable_parameters *= dim.value
    print(variable_parameters)
    total_parameters += variable_parameters
print(total_parameters)

(2, 2, 1, 64)
4
2
2
1
64
256
(64,)
1
64
64
(3, 3, 64, 128)
4
3
3
64
128
73728
(128,)
1
128
128
(4, 4, 128, 128)
4
4
4
128
128
262144
(128,)
1
128
128
(128, 128)
2
128
128
16384
(128,)
1
128
128
(128, 128)
2
128
128
16384
(128,)
1
128
128
(128, 2)
2
128
2
256
(2,)
1
2
2
369730


In [19]:
pool3.shape

TensorShape([Dimension(None), Dimension(1), Dimension(1), Dimension(128)])

In [20]:
# None, as in, this is not yet defined, there could be any number of them input. 
# 2, as in, there are two elements in the one-hot vector

labels = tf.placeholder(tf.int32, [None, 2], name="labels")

In [21]:
# This loss is the elementwise loss
#loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)

with tf.name_scope("Loss"):
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=predictions))

"""with tf.name_scope("Loss"):
    loss = tf.losses.mean_squared_error(labels=labels, predictions=predictions)"""

with tf.name_scope("Optimizer"):
    train = tf.train.AdamOptimizer(learning_rate=0.001, name="Adam").minimize(loss)

In [22]:
with tf.name_scope("Error"):
    error = tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(labels, 1), tf.argmax(predictions, 1)), tf.float32), name="Mean")

## Define an error rate to evaluate model performance


In [23]:
tf.summary.scalar("Loss", loss)
tf.summary.scalar("Error", error)

<tf.Tensor 'Error_1:0' shape=() dtype=string>

In [24]:
number_of_batches = int(N_TRAIN / BATCH_SIZE)
print(number_of_batches)

1250


In [25]:
sess = tf.Session()

In [26]:
sess.run(tf.global_variables_initializer())

In [27]:
for epoch in range(1):
    #randomise the data
    gb, lbl  = get_randomised_batches(train_gameboards, train_labels)
    
    for i in range(N_BATCHES):
        first_index = i*BATCH_SIZE
        second_index = (i*BATCH_SIZE) + BATCH_SIZE
        batch = gb[first_index:second_index]
        batchLabels = lbl[first_index:second_index]

        #print(np.asarray(batch).shape)

        if i % 100 == 0:
            Error, Loss = sess.run([error, loss], feed_dict={gameboards:batch, labels:batchLabels, keep_prob:1.0})
            print("{}, Loss: {}, \tError: {}%".format(i, Loss, Error*100))
        sess.run(train, feed_dict={gameboards:batch, labels:batchLabels, keep_prob:0.5})


0, Loss: 0.6943117380142212, 	Error: 56.25%
100, Loss: 0.651033341884613, 	Error: 39.0625%
200, Loss: 0.6728786826133728, 	Error: 41.40625%
300, Loss: 0.658026933670044, 	Error: 42.578125%
400, Loss: 0.6569036841392517, 	Error: 42.1875%
500, Loss: 0.6429314613342285, 	Error: 34.765625%
600, Loss: 0.6106640100479126, 	Error: 35.546875%
700, Loss: 0.6496871709823608, 	Error: 40.625%
800, Loss: 0.6664119958877563, 	Error: 36.71875%
900, Loss: 0.6470652222633362, 	Error: 36.71875%
1000, Loss: 0.6233158111572266, 	Error: 33.59375%
1100, Loss: 0.6245087385177612, 	Error: 36.328125%
1200, Loss: 0.6418462991714478, 	Error: 37.5%


In [28]:
val = []

for i in validation_gameboards:
    val.append(np.reshape(i, (100)))

In [29]:
#test = []

#for i in test_gameboards:
    #test.append(np.reshape(i, (100)))

In [30]:
Error = sess.run(error, feed_dict={gameboards:val, labels:validation_labels, keep_prob:1.0})
print(Error)

0.385191


In [31]:
#Error = sess.run(error, feed_dict={gameboards:test, labels:test_labels, keep_prob:1.0})
#print(Error)

In [32]:
saver = tf.train.Saver()
saver.save(sess, "nn_model\\")

'nn_model\\'

In [33]:
for i in range(100):
    r = sess.run(predictions, feed_dict={gameboards:[val[i]], keep_prob:1.0})
    print(r, validation_labels[i])

[[ 0.20382451 -0.09565792]] [0 1]
[[ 0.18464547 -0.08911093]] [0 1]
[[ 0.26652789 -0.12469832]] [0 1]
[[ 0.3028461  -0.14244349]] [1 0]
[[ 0.17877287 -0.07484733]] [1 0]
[[ 0.82947767 -0.48222473]] [1 0]
[[ 0.05582605 -0.01011826]] [0 1]
[[-0.00897887  0.0495826 ]] [0 1]
[[-0.04630198  0.09065548]] [0 1]
[[ 0.23430805 -0.10631594]] [0 1]
[[ 0.00642218  0.05507131]] [0 1]
[[ 0.01595702  0.0363302 ]] [0 1]
[[-0.07293038  0.11721419]] [0 1]
[[-0.32008162  0.40287516]] [0 1]
[[-0.11103632  0.17234212]] [0 1]
[[ 0.06609468 -0.01888115]] [1 0]
[[-0.01011379  0.04852433]] [1 0]
[[ 0.57031024 -0.2735377 ]] [1 0]
[[ 0.19957274 -0.0944072 ]] [0 1]
[[ 0.18103392 -0.08464561]] [0 1]
[[-0.54026276  0.69360888]] [0 1]
[[ 0.17231245 -0.0796057 ]] [1 0]
[[ 0.10663161 -0.0580479 ]] [0 1]
[[ 0.04098496 -0.00874101]] [0 1]
[[ 0.15553157 -0.07840641]] [0 1]
[[ 0.14020337 -0.06650194]] [1 0]
[[ 0.18814886 -0.08472127]] [1 0]
[[ 0.18670744 -0.08604512]] [1 0]
[[ 0.21629985 -0.09472647]] [1 0]
[[ 0.23282041 

In [34]:
print(r)

[[ 0.14156681 -0.06316273]]
